In [1]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import string

/Users/hungtran/Documents/Anaconda/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Read Data

In [2]:
df = pd.read_csv('data/Danh_gia.csv', encoding='utf-8')
df.head()

,id,ma_khach_hang,noi_dung_binh_luan,ngay_binh_luan,gio_binh_luan,so_sao,ma_san_pham
0,1,443,"SỬ DỤNG DỄ DÀNG, RẤT THOẢI MÁI, THƯ GIÃN TỘT ĐỘ.",29/04/2023,17:06,5,308500015
1,2,1030,"Sử dụng dễ dãng,rất thoải mái,thư giãn tột độ",30/04/2023,15:04,5,308500015
2,3,689,Mình rất thích hasaki va sp tẩy trang này,30/04/2023,18:34,5,422216594
3,4,2519,Sản phẩm có khả năng làm sạch tốt. Lớp trang đ...,17/07/2022,13:48,5,204100075
4,5,402,"Sữa rửa mặt tốt,sạch mụn,mịn da,đáng mua nha",15/04/2023,23:04,5,422208977


### Preprocessing 

#### Check null

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21575 entries, 0 to 21574
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  21575 non-null  int64 
 1   ma_khach_hang       21575 non-null  int64 
 2   noi_dung_binh_luan  20674 non-null  object
 3   ngay_binh_luan      21575 non-null  object
 4   gio_binh_luan       21575 non-null  object
 5   so_sao              21575 non-null  int64 
 6   ma_san_pham         21575 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.2+ MB


In [4]:
#drop null:
df.dropna(inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20674 entries, 0 to 21574
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  20674 non-null  int64 
 1   ma_khach_hang       20674 non-null  int64 
 2   noi_dung_binh_luan  20674 non-null  object
 3   ngay_binh_luan      20674 non-null  object
 4   gio_binh_luan       20674 non-null  object
 5   so_sao              20674 non-null  int64 
 6   ma_san_pham         20674 non-null  int64 
dtypes: int64(4), object(3)
memory usage: 1.3+ MB


### Check duplicated

In [6]:
#check duplicated
df.duplicated().sum()

0

### Preprocess text

In [ ]:
#LOAD EMOJICON
file = open('files/emojicon.txt', 'r', encoding="utf8")
emoji_lst = file.read().split('\n')
emoji_dict = {}
for line in emoji_lst:
    key, value = line.split('\t')
    emoji_dict[key] = str(value)
file.close()
#################
#LOAD TEENCODE
file = open('files/teencode.txt', 'r', encoding="utf8")
teen_lst = file.read().split('\n')
teen_dict = {}
for line in teen_lst:
    key, value = line.split('\t')
    teen_dict[key] = str(value)
file.close()
###############
#LOAD TRANSLATE ENGLISH -> VNMESE
file = open('files/english-vnmese.txt', 'r', encoding="utf8")
english_lst = file.read().split('\n')
english_dict = {}
for line in english_lst:
    key, value = line.split('\t')
    english_dict[key] = str(value)
file.close()
################
#LOAD wrong words
file = open('files/wrong-word.txt', 'r', encoding="utf8")
wrong_lst = file.read().split('\n')
file.close()
#################
#LOAD STOPWORDS
file = open('files/vietnamese-stopwords.txt', 'r', encoding="utf8")
stopwords_lst = file.read().split('\n')
file.close()

In [ ]:
#Process text:
def process_text(text, emoji_dict, teen_dict, wrong_lst):
    document = text.lower()
    document = document.replace("’",'')
    document = regex.sub(r'\.+', ".", document)
    new_sentence =''
    for sentence in sent_tokenize(document):
        # if not(sentence.isascii()):
        ###### CONVERT EMOJICON
        sentence = ''.join(emoji_dict[word]+' ' if word in emoji_dict else word for word in list(sentence))
        ###### CONVERT TEENCODE
        sentence = ' '.join(teen_dict[word] if word in teen_dict else word for word in sentence.split())
        ###### DEL Punctuation & Numbers
        pattern = r'(?i)\b[a-záàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ]+\b'
        sentence = ' '.join(regex.findall(pattern,sentence))
        # ...
        ###### DEL wrong words
        sentence = ' '.join('' if word in wrong_lst else word for word in sentence.split())
        new_sentence = new_sentence+ sentence + '. '
    document = new_sentence
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    #...
    return document

In [ ]:
df.noi_dung_binh_luan = df.noi_dung_binh_luan.apply(lambda x: process_text(x, emoji_dict, teen_dict, wrong_lst))

In [ ]:
df.noi_dung_binh_luan

,noi_dung_binh_luan
0,sử dụng dễ dàng rất thoải mái thư giãn tột độ.
1,sử dụng dễ dãng rất thoải mái thư giãn tột độ.
2,mình rất thích hasaki sản phẩm tẩy này.
3,sản phẩm có khả năng làm sạch tốt. lớp điểm tr...
4,sữa rửa mặt tốt sạch mụn mịn đáng .
...,...
21570,dùng sản phẩm này bị ghiền luôn hạt siêu mịn m...
21571,nhiều lần ở đây rồi ạ sản phẩm giá rẻ hàng nữa...
21572,sản phẩm có một số hạn chế không thực sự ưng ý.
21573,dùng này từ năm trước sáng trắng rất ưng ý.


In [ ]:
# Chuẩn hóa unicode tiếng việt
def loaddicchar():
    uniChars = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    unsignChars = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"

    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

# Đưa toàn bộ dữ liệu qua hàm này để chuẩn hóa lại
def covert_unicode(txt):
    dicchar = loaddicchar()
    return regex.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)

In [ ]:
df.noi_dung_binh_luan = df.noi_dung_binh_luan.apply(lambda x: covert_unicode(x))

In [ ]:
df.noi_dung_binh_luan

,noi_dung_binh_luan
0,sử dụng dễ dàng rất thoải mái thư giãn tột độ.
1,sử dụng dễ dãng rất thoải mái thư giãn tột độ.
2,mình rất thích hasaki sản phẩm tẩy này.
3,sản phẩm có khả năng làm sạch tốt. lớp điểm tr...
4,sữa rửa mặt tốt sạch mụn mịn đáng .
...,...
21570,dùng sản phẩm này bị ghiền luôn hạt siêu mịn m...
21571,nhiều lần ở đây rồi ạ sản phẩm giá rẻ hàng nữa...
21572,sản phẩm có một số hạn chế không thực sự ưng ý.
21573,dùng này từ năm trước sáng trắng rất ưng ý.


In [ ]:
def process_special_word(text):
    # Danh sách các từ tiêu cực
    special_words = ['không', 'chẳng', 'chả', 'kém', 'tệ', 'xấu', 'tồi', 'thất',
                     'ngu', 'dốt', 'dở', 'mệt', 'sai', 'ghét', 'buồn', 'lo', 'chán',
                     'thất vọng', 'nhục', 'xấu hổ', 'đau', 'thất bại', 'khóc',
                     'giận', 'bực']
    new_text = ''
    text_lst = text.split()
    i = 0

    while i <= len(text_lst) - 1:
        word = text_lst[i]
        # Kiểm tra nếu từ hiện tại nằm trong danh sách từ tiêu cực
        if word in special_words:
            next_idx = i + 1
            if next_idx <= len(text_lst) - 1:
                # Ghép từ tiêu cực với từ kế tiếp
                word = word + '_' + text_lst[next_idx]
            i = next_idx + 1  # Bỏ qua từ kế tiếp đã được ghép
        else:
            i = i + 1  # Chuyển sang từ tiếp theo
        new_text = new_text + word + ' '

    return new_text.strip()


In [ ]:
df.noi_dung_binh_luan = df.noi_dung_binh_luan.apply(lambda x: process_special_word(x))

In [ ]:
df.noi_dung_binh_luan

,noi_dung_binh_luan
0,sử dụng dễ dàng rất thoải mái thư giãn tột độ.
1,sử dụng dễ dãng rất thoải mái thư giãn tột độ.
2,mình rất thích hasaki sản phẩm tẩy này.
3,sản phẩm có khả năng làm sạch tốt. lớp điểm tr...
4,sữa rửa mặt tốt sạch mụn mịn đáng .
...,...
21570,dùng sản phẩm này bị ghiền luôn hạt siêu mịn m...
21571,nhiều lần ở đây rồi ạ sản phẩm giá rẻ hàng nữa...
21572,sản phẩm có một số hạn chế không_thực sự ưng ý.
21573,dùng này từ năm trước sáng trắng rất ưng ý.


In [ ]:
import re
# Hàm để chuẩn hóa các từ có ký tự lặp
def normalize_repeated_characters(text):
    # Thay thế mọi ký tự lặp liên tiếp bằng một ký tự đó
    # Ví dụ: "lònggggg" thành "lòng", "thiệtttt" thành "thiệt"
    return re.sub(r'(.)\1+', r'\1', text)

# Áp dụng hàm chuẩn hóa cho văn bản
# print(normalize_repeated_characters(example))

In [ ]:
df.noi_dung_binh_luan = df.noi_dung_binh_luan.apply(lambda x: normalize_repeated_characters(x))

In [ ]:
df.noi_dung_binh_luan

,noi_dung_binh_luan
0,sử dụng dễ dàng rất thoải mái thư giãn tột độ.
1,sử dụng dễ dãng rất thoải mái thư giãn tột độ.
2,mình rất thích hasaki sản phẩm tẩy này.
3,sản phẩm có khả năng làm sạch tốt. lớp điểm tr...
4,sữa rửa mặt tốt sạch mụn mịn đáng .
...,...
21570,dùng sản phẩm này bị ghiền luôn hạt siêu mịn m...
21571,nhiều lần ở đây rồi ạ sản phẩm giá rẻ hàng nữa...
21572,sản phẩm có một số hạn chế không_thực sự ưng ý.
21573,dùng này từ năm trước sáng trắng rất ưng ý.


In [ ]:
def process_postag_thesea(text):
    new_document = ''
    for sentence in sent_tokenize(text):
        sentence = sentence.replace('.','')
        ###### POS tag
        lst_word_type = ['N','Np','A','AB','V','VB','VY','R']
        # lst_word_type = ['A','AB','V','VB','VY','R']
        sentence = ' '.join( word[0] if word[1].upper() in lst_word_type else '' for word in pos_tag(process_special_word(word_tokenize(sentence, format="text"))))
        new_document = new_document + sentence + ' '
    ###### DEL excess blank space
    new_document = regex.sub(r'\s+', ' ', new_document).strip()
    return new_document

In [ ]:
df.noi_dung_binh_luan = df.noi_dung_binh_luan.apply(lambda x: process_postag_thesea(x))

In [ ]:
df.noi_dung_binh_luan

,noi_dung_binh_luan
0,sử_dụng dễ_dàng rất thoải_mái thư_giãn
1,sử_dụng dễ dãng rất thoải_mái thư_giãn
2,rất thích hasaki sản_phẩm tẩy
3,sản_phẩm có khả_năng làm sạch tốt lớp điểm mắt...
4,sữa rửa mặt tốt sạch mụn mịn đáng
...,...
21570,dùng sản_phẩm bị ghiền luôn hạt mịn mùi thơm d...
21571,nhiều lần sản_phẩm giá rẻ hàng nữa chớ yêu
21572,sản_phẩm có một_số hạn_chế không_thực_sự
21573,dùng năm trước sáng trắng rất ưng_ý


In [ ]:
def remove_stopword(text, stopwords):
    ###### REMOVE stop words
    document = ' '.join('' if word in stopwords else word for word in text.split())
    #print(document)
    ###### DEL excess blank space
    document = regex.sub(r'\s+', ' ', document).strip()
    return document

In [ ]:
df.noi_dung_binh_luan = df.noi_dung_binh_luan.apply(lambda x: remove_stopword(x, stopwords_lst))

In [ ]:
df.noi_dung_binh_luan

,noi_dung_binh_luan
0,dễ_dàng thoải_mái thư_giãn
1,dãng thoải_mái thư_giãn
2,thích hasaki tẩy
3,khả_năng sạch tốt lớp mắt môi_chóng loại_bỏ bô...
4,sữa rửa mặt tốt sạch mụn mịn
...,...
21570,ghiền hạt mịn mùi thơm dễ_chịu sạch
21571,giá rẻ hàng yêu
21572,hạn_chế không_thực_sự
21573,trắng ưng_ý


In [ ]:
#Them label

In [ ]:
df['label'] = df.so_sao.apply(lambda x: 'good' if x > 3 else 'bad' if x < 3 else 'neutral')

In [ ]:
df['label']

,label
0,good
1,good
2,good
3,good
4,good
...,...
21570,good
21571,good
21572,bad
21573,good


In [ ]:
df.head()

,id,ma_khach_hang,noi_dung_binh_luan,ngay_binh_luan,gio_binh_luan,so_sao,ma_san_pham,label
0,1,443,dễ_dàng thoải_mái thư_giãn,29/04/2023,17:06,5,308500015,good
1,2,1030,dãng thoải_mái thư_giãn,30/04/2023,15:04,5,308500015,good
2,3,689,thích hasaki tẩy,30/04/2023,18:34,5,422216594,good
3,4,2519,khả_năng sạch tốt lớp mắt môi_chóng loại_bỏ bô...,17/07/2022,13:48,5,204100075,good
4,5,402,sữa rửa mặt tốt sạch mụn mịn,15/04/2023,23:04,5,422208977,good


In [ ]:
#Check imbalance

In [ ]:
df.label.value_counts()

,count
label,
good,18650
bad,1045
neutral,979


### **Export preprocessed data**

In [ ]:
df.to_csv('data/Preprocess_San_pham.csv', index=False)